# hs-code-nlp-classifier-cl

## Proyecto

### Contexto

- El Servicio Nacional de Aduanas de Chile aporta cerca del 30% de los impuestos que financian el desarrollo del país. 
- La correcta clasificación (código arancelario) es clave para: a) Cálculo de Aranceles, b) Aplicación de Tratados de Comercio, y  c) Cumplimiento Normativo.
- Estándar Global con Aplicación Local: se utiliza el Sistema Armonizado (HS), un estándar mundial (95% de adherencia) que requiere de ajustes y jurisprudencia local específicos de Chile.

### Problema

- El Desafío de Clasificar: un proceso manual, específico y dinámico.
- Proceso manual e intenso (tedioso) realizado por operadores humanos (conocimiento experto, sistemas informáticos para registro) de Agencias de Aduana. 

### Relevancia

- El error genera riesgo de multas/pérdida de tiempo en correcciones para las Agencias de Aduanas y Operadores de Comercio Exterior, y pérdida de recaudación para Aduanas Chile, afectando la calidad del servicio de las agencias. 
- Incumplimiento normativo (las multas afectan a la agencia de aduanas), también puede existir un tiempo de clasificación alto, afectando al cliente y al equipo humano de trabajo.

## Configuraciones

In [ ]:
import sys
import json
import inspect
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud
import torch
from torch.utils.data import Dataset as TorchDataset
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, classification_report, confusion_matrix

import transformers
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed,
)
import gc
import os


In [ ]:
path_proyecto = Path.cwd()
path_src = path_proyecto / "src"
if str(path_src) not in sys.path:
    sys.path.append(str(path_src))


In [ ]:
%load_ext autoreload
%autoreload 2
from src.utils import *
from src.csv import *


In [ ]:
crear_carpetas()
DIR_RAW_DATA = obtener_path("raw")
DIR_IMAGES = obtener_path("images")
DIR_OUTPUTS = obtener_path("outputs")
DIR_MODELS = obtener_path("models")
DIR_LOG = obtener_path("logs")


In [ ]:
SEED = obtener_seed()
print(f"[INFO] Seed usada: {SEED}")


## Dataset

### Carga Datos Etiquetados

In [ ]:
print("[INFO] Dispositivo:")
print(f" - torch.backends.mps.is_available(): {torch.backends.mps.is_available()}")
print(f" - torch.backends.mps.is_built(): {torch.backends.mps.is_built()}")
device = (
    torch.device("mps") if torch.backends.mps.is_available()
    else torch.device("cuda") if torch.cuda.is_available()
    else torch.device("cpu")
)
print(f" - torch version: {torch.__version__}")
print(f" - device seleccionado: {device}")


In [ ]:
CVV_FILENAME = "mercancias20260111_004347.csv"

raw_df = cargar_csv(Path(DIR_RAW_DATA, CVV_FILENAME))


### Exploración

#### Características

**Columnas:**

In [ ]:
print(f"[INFO] Columnas del Dataset: {raw_df.columns}")


**Muestra de registros**:

In [ ]:
raw_df.sample(10, random_state=SEED)


**WordCloud**:

In [ ]:
wc_cache_image = "wordcloud-raw-data.png"
wc_cache_path = Path(DIR_IMAGES, wc_cache_image)

if wc_cache_path.exists():
    img = Image.open(wc_cache_path)
else:
    text = " ".join(
        raw_df["mercancia_descripcion"]
        .fillna("")
        .astype(str)
        .tolist()
    )
    wc = WordCloud(width=1000, height=500, background_color="white").generate(text)
    img = wc.to_image()
    img.save(wc_cache_path)

plt.figure(figsize=(12,6))
plt.imshow(np.array(img), interpolation="bilinear")
plt.axis("off")
plt.show()


**Notas**:
- La nube muestra que las descripciones están dominadas por términos de uso y destino (“PARA”, “USO”, “PARTE”, “DE”), además de categorías de producto frecuentes como “VEHICULO”, “ACERO”, “SANDVIK”, “TEJIDO”, “PIEZA”. Esto sugiere un lenguaje muy estandarizado y orientado a “para X”, lo que puede introducir stopwords específicas del dominio (ej. “para”, “de”, “uso”) que quizá conviene filtrar en modelos VCM para resaltar términos más discriminantes.

#### Datos no válidos

In [ ]:
desc_vacias = (
    raw_df["mercancia_descripcion"]
    .fillna("")
    .str.strip()
    .eq("")
    .sum()
)
print("[INFO] Registros con descripción vacía:", desc_vacias)


In [ ]:
# Eliminar registros con descripción vacía o solo espacios
data_df = raw_df[
    raw_df["mercancia_descripcion"]
    .fillna("")
    .str.strip()
    .ne("")
].copy()

print("[INFO] Info dataset:")
print(" - Registros totales:", len(raw_df))
print(" - Registros tras limpieza:", len(data_df))


In [ ]:
data_df["len_descripcion"] = (
    data_df["mercancia_descripcion"]
    .fillna("")
    .astype(str)
    .str.len()
)


In [ ]:
data_df["len_descripcion"].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])


**Notas**:
- Se analizaron **5,125,025** descripciones.
- La longitud promedio es **~53 caracteres** y la mediana es **50**, lo que indica textos relativamente cortos.
- El **75%** de las descripciones tiene **≤63 caracteres**.
- El **90%** está bajo **80 caracteres**, y el **95%** bajo **90**.
- El **99%** no supera **116 caracteres**; el máximo observado es **180**.

**Conclusión:** la mayoría de las descripciones son muy breves. Para BERT, un `max_length` de **128** sería suficiente para cubrir casi todo el universo sin truncar.


#### Frecuencias de Clases

In [ ]:
total_filas = data_df["partida_arancelaria_codigo"].notna().sum()


In [ ]:
# Frecuencia de todas las clases

raw_freq_df = data_df["partida_arancelaria_codigo"].value_counts()
raw_freq_df.head(20)


In [ ]:
# Distribución de clases (frecuencia)

raw_freq_df.describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])


**notas**
- Se identifican **5.597 clases** distintas.
- La mediana es 51: la mitad de las clases tiene 51 registros o menos.
- El **25% de las clases** tiene **9 registros o menos**, lo que indica presencia de muchas clases poco representadas.
- El 75% tiene ≤292 registros.
- Solo el 10% supera ~1.381 registros.
- El 1% supera ~17.474 registros.
- El máximo es 126.200, alto contraste.
- Conclusiones: 
  - Hay muchas clases con muy pocos ejemplos y pocas clases dominantes. 
  - Es posible que se deba limitar clases más representadas o agrupar clases poco frecuentes en `OTHER`.
- Para modelar, se puede explorar algunas técnicas:
  - Limitar a las clases más frecuentes
  - Definir etiqueta OTHER para agrupar las poco representadas
  - Aplicar técnicas de balanceo.

In [ ]:
# Análisis de cobertura por top N clases

total = raw_freq_df.sum()

for n in [5, 10, 20, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]:
    topn = raw_freq_df.head(n)
    rest = raw_freq_df.iloc[n:]
    prop_top = topn.sum() / total * 100
    prop_rest = rest.sum() / total * 100
    print(f"Top {n:4d}: {topn.sum():>10} ({prop_top:6.2f}%) | Resto: {rest.sum():>10} ({prop_rest:6.2f}%)")


**Notas**:
- La distribución está fuertemente concentrada: **Top 5** ya cubre **10%** del total.
- Con **Top 10** se cubre **18%**, menos de una quinta parte del universo.
- Al llegar a **Top 50**, se cubre **42%**; es decir, más de la mitad sigue en “resto”.
- El **Top 100** cubre **55%** y el **Top 200** alcanza **67%**.
- Para superar el **80%** de cobertura se necesitan alrededor de **400 clases** (79%) a **500 clases** (82%).
- Para cobertura alta, el número de clases objetivo debe ser grande (≈400–500 para >80%). 
- Para un modelo más acotado, conviene usar `OTHER` para el resto o limitarse a un Top‑N más pequeño con una cobertura aceptada.


In [ ]:
# Muestra de clases más representadas

n = 20
raw_clases_top_n = (
    data_df["partida_arancelaria_codigo"]
    .value_counts()
    .head(n)
    .reset_index()
    .rename(columns={"index": "partida_arancelaria_codigo", "partida_arancelaria_codigo": "frecuencia"})
)

raw_clases_top_n["porcentaje_total"] = (raw_clases_top_n["count"] / total_filas) * 100
raw_clases_top_n["porcentaje_total"] = raw_clases_top_n["porcentaje_total"].map(lambda x: f"{x:.2f}%")

raw_clases_top_n


In [ ]:
# Clases con muy baja representación (umbral)

umbral = 500
clases_poco_representadas = raw_freq_df[raw_freq_df < umbral]

print("[INFO] Clases con < umbral() :", len(clases_poco_representadas))
clases_poco_representadas.head(10)


**Notas**
- El resultado indica que 4.534 clases tienen menos de 500 registros (según umbral). 
- En el extracto se ve que muchas están justo cerca del límite (490–499), lo que sugiere una larga cola de clases poco representadas.

**Criterio máxima curvatura sobre cobertura vs umbral**:

In [ ]:
# Umbrales automáticos en log-escala para cubrir bien rangos bajos/medios

print("[INFO] Selección automática de umbral - Criterio máxima curvatura sobre cobertura vs umbral")
min_t = max(1, int(raw_freq_df.min()))
max_t = int(raw_freq_df.max())
thresholds = np.unique(np.round(np.logspace(np.log10(min_t), np.log10(max_t), 40)).astype(int))

coverage = []
for t in thresholds:
    keep = raw_freq_df[raw_freq_df >= t]
    coverage.append(keep.sum() / total_filas * 100)

# Curvatura numérica
x = np.log10(thresholds.astype(float))
y = np.array(coverage)

x_norm = (x - x.min()) / (x.max() - x.min())
y_norm = (y - y.min()) / (y.max() - y.min())

curvature = np.abs(np.gradient(np.gradient(y_norm, x_norm), x_norm))
knee_idx = int(np.argmax(curvature))
knee_t = thresholds[knee_idx]
knee_cov = coverage[knee_idx]

print(" - Umbral recomendado (knee):", knee_t)
print(" - Cobertura en ese umbral:", f"{knee_cov:.2f}%")
print(" - Clases que cumplen umbral:", int((raw_freq_df >= knee_t).sum()))


**Criterio mínimo N que alcance 80–90% del total**:

In [ ]:
# Análisis de cobertura por porcentaje total

targets = [0.50, 0.60, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]
cum = raw_freq_df.cumsum() / raw_freq_df.sum()
for target in targets:
    n = (cum <= target).sum() + 1
    print(f"% Representación del {target:.2f} total: {n} clases")


**Criterio Mínimo por split**: asegurar al menos X ejemplos en test.

In [ ]:
# Umbral basado en número de muestras en test set

test_size = 0.2
min_tests = [60, 80, 120, 150, 250, 300, 400, 500, 600, 700, 800]

for min_test in min_tests:
    min_muestras = int(min_test / test_size)
    classes_keep = raw_freq_df[raw_freq_df >= min_muestras].index
    print(f"min_test {min_test:4d} -> min_muestras {min_muestras:5d} -> clases {len(classes_keep)}")


**Notas**:
- mínimo por split (min_test) vs clases retenidas
  - A mayor `min_test`, mayor `min_muestras` requerido por clase, por lo que disminuye el número de clases que cumplen el umbral.
  - Con **min_test=60–150** (300–750 muestras por clase) aún se conservan **~860–1,385 clases**, un conjunto amplio.
  - Con **min_test=250–400** (1,250–2,000 por clase) el conjunto baja a **~432–609 clases**, un rango más manejable.
  - Con **min_test=600–800** (3,000–4,000 por clase) quedan **~222–299 clases**, priorizando estabilidad y calidad de entrenamiento.

**Conclusión**: 
- Para un clasificador con alta precisión y clases bien representadas, un rango útil sería **min_test 250–500** (≈350–600 clases). 
- Si se requiere más estabilidad por clase, se puede aumentar a **min_test 600+**.


In [ ]:
# Proporción acumulada de frecuencias:
raw_freq_df = raw_freq_df.sort_values(ascending=False)
cum = raw_freq_df.cumsum() / raw_freq_df.sum()

# Clases que cubren x% del total (umbral de cobertura acumulada
cutoffs = [0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
for cutoff in cutoffs:
    classes_keep = cum[cum <= cutoff].index
    print(f"Cutoff {cutoff:.2f}: {len(classes_keep)} clases")


**Notas**;

- **Interpretación de clases necesarias por cobertura (cutoff)**:
  - **50% de cobertura** requiere **74 clases**.
  - **60%** requiere **132 clases**.
  - **70%** requiere **234 clases**.
  - **75%** requiere **312 clases**.
  - **80%** requiere **420 clases**.
  - **85%** requiere **572 clases**.
  - **90%** requiere **816 clases**.
  - **95%** requiere **1,295 clases**.
- **Conclusión**: 
  - La cobertura crece rápido al inicio, pero a partir de 80% el número de clases aumenta de forma acelerada.
  - Confirma una distribución altamente dispersa. 
  - 70–85% suele ser un buen rango de compromiso.


**Umbral vs Clases y Cobertura**

In [ ]:
# Explorar umbrales
thresholds = [100, 200, 300, 400, 500, 600, 800, 1000, 1500, 2000, 3000, 4000, 5598]

n_classes = []
coverage = []

for t in thresholds:
    keep = raw_freq_df[raw_freq_df >= t]
    n_classes.append(len(keep))
    coverage.append(keep.sum() / total_filas * 100)

fig, ax1 = plt.subplots(figsize=(12,4))
ax1.plot(thresholds, n_classes, marker="o", label="Clases cumplen umbral")
ax1.set_xlabel("min_muestras por clase")
ax1.set_ylabel("Número de clases")
ax1.grid(True, alpha=0.3)

ax2 = ax1.twinx()
ax2.plot(thresholds, coverage, marker="s", color="orange", label="Cobertura (%)")
ax2.set_ylabel("Cobertura del total (%)")

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc="best")

plt.title("Selección de umbral: clases vs cobertura")
plt.show()


**Notas:**
- A medida que aumenta `min_muestras` por clase, disminuye el `número de clases` que cumplen el umbral.
- La **cobertura total** cae de forma gradual: 
  - Con umbrales bajos se mantiene alta, pero se reduce al exigir más ejemplos por clase.
- Se observa un **trade‑off claro**: 
  - Umbrales bajos permiten muchas clases pero con menor calidad/estabilidad
  - Umbrales altos priorizan clases bien representadas pero reducen la cobertura.
- El “punto de equilibrio” visual parece estar entre **1,000 y 2,000 muestras por clase**, donde aún hay cientos de clases y una cobertura razonable.


### Definiciones

**Información**:
- Registros totales: 5.136.897
- Registros tras limpieza: 5.125.025 (se excluyen, por ejemplo, descripciones vacías: 11.872)
- **Variable objetivo**: partida_arancelaria_codigo (códigos tipo 8 dígitos, p. ej. 30061059)
- **Desbalance crítico**: con un umbral de 500 muestras por clase, hay 4.534 clases bajo ese mínimo (colas largas).
- La distribución está dominada por pocas clases (las más frecuentes superan 100k ocurrencias), mientras muchas clases aparecen pocas centenas de veces.

En este contexto, la decisión clave para maximizar éxito (accuracy/F1) es definir cuántas clases se intentará predecir (y con qué soporte por clase).


**Criterios para definir Umbrales**:
1.	Corte por cobertura acumulada (top-K clases que explican X% del total):
    - 0,70 -> 234 clases
    - 0,75 -> 312 clases
    - 0,80 -> 420 clases
    - 0,85 -> 572 clases
    - 0,90 -> 816 clases
    - 0,95 -> 1295 clases

2.	Corte por mínimo de muestras en test, asumiendo test_size = 0.2 (para que cada clase tenga suficiente evidencia en evaluación):
  - **min_test 150** -> min_train+test 750 -> 859 clases
  - **min_test 400** -> min_train+test 2000 -> 430 clases
  - **min_test 800** -> min_train+test 4000 -> 222 clases

3. **Clasificador `SVM`**
   - Propuesta: ~300 clases (rango 234–312)
     - **Opción principal** (balanceada): 312 clases (cubre ~75% del dataset).
     - **Opción alternativa** (menos costo): 234 clases (cubre ~70%).
     - **Opción alternativa por soporte estadístico**: ~300 clases si se fija `min_muestras` ≈ 3000 (derivado del análisis min_test/0.2).
   - **Justificación técnica**: en multiclase grande, SVM (aun lineal) tiende a degradar más rápido por solapamiento semántico y ruido en descripciones, y además el costo de one-vs-rest crece con #clases. Reducir a ~300 clases suele dar un salto grande en precisión y estabilidad.

4) **Clasificador `BERT`**
   - Propuesta: ~430–572 clases (recomendación operativa: 430)
     - **Opción principal** (mejor trade-off para “alto éxito”): ~430 clases, alineado con `min_muestras` ≈ 2000 (equivale a exigir ~400 ejemplos mínimos en test con test_size=0.2).
     - **Opción alternativa**: 572 clases (cubre ~85% del dataset), pero es más exigente: aumentan confusiones entre códigos cercanos y se requiere mejor normalización del texto + más cuidado con entrenamiento (scheduler, epochs, class weights/focal, etc.).
   - **Justificación técnica**: BERT captura mejor la semántica y suele sostener desempeño con más clases que SVM, si se incluyen muchas clases raras, la métrica se hunde por falta de ejemplos y por descripciones ambiguas.

**Decisión Final**: 
- Foco de clasificador: “alto éxito”
- **SVM**: entrenar sobre 312 clases (cobertura ~75%).
- **BERT**: entrenar sobre 430 clases (mínimo ~2000 muestras por clase; evaluación más estable).
- Esto produce dos modelos comparables, ambos “de alta probabilidad de éxito”, pero con BERT cubriendo más taxonomía sin sacrificar tanto la calidad.


## Arquitectura

```text
    Entrada: descripción del producto
                ↓
[Filtro IN/OUT — BERT binario compartido]
                ↓ 
   ┌───────────────────────────────┐
   ↓                               ↓
[SVM multiclase]           [BERT multiclase] 
        ↓                          ↓
   Código HS                  Código HS
        ↓                          ↓
            [Revisión Expeta]
```

## Modelo Filtro IN/OUT

In [ ]:
# Modelo base para el filtro binario

DF_NAME = "data_df" 
COL_TEXT = "mercancia_descripcion"  # columna con la descripción (texto)
COL_CODE = "partida_arancelaria_codigo"  # columna con HS code (string/num)
MODEL_NAME = "bert-base-multilingual-cased"  # alternativa: "roberta-base", "xlm-roberta-base"
MAX_LENGTH = 64
MAX_ROWS_FOR_FILTER = 400000  # ajuste según RAM/tiempo

# Split: se hará split en dos etapas: train vs temp, luego val vs test
TEST_SIZE = 0.2
N_PER_CLASS = 2000
VAL_SIZE = 0.10

# IN por TOP_K clases más frecuentes
TOP_K_IN = 430  # 312 para SVM, 430 para BERT.

# Entrenamiento 
BATCH_TRAIN = 16
BATCH_EVAL = 32
GRADIENT_ACCUMULATION_STEPS = 4

LR = 2e-5
EPOCHS = 2
WEIGHT_DECAY = 0.01
WARMUP_RATIO = 0.06

TOKENIZERS_PARALLELISM = False
set_seed(SEED)


Construcción del set de clases "IN" (top-K o por cobertura)

In [ ]:
def preprocesar_texto(text: str) -> str:
    if text is None:
        return ""
    text = str(text)
    text = text.replace("\n", " ").replace("\r", " ")
    text = " ".join(text.split())
    return text


In [ ]:
print(f"[INFO] TOP_K_IN={TOP_K_IN}")
filename_clases_in = "clases_ins.csv"
path_clases_in = Path(DIR_LOG, filename_clases_in)
df_in_top_k = globals()[DF_NAME].copy()

# Normaliza tipos
df_in_top_k[COL_TEXT] = df_in_top_k[COL_TEXT].astype(str).str.strip()
df_in_top_k[COL_TEXT] = df_in_top_k[COL_TEXT].apply(preprocesar_texto)
df_in_top_k[COL_CODE] = df_in_top_k[COL_CODE].astype(str).str.strip()
df_in_top_k = df_in_top_k[df_in_top_k[COL_TEXT].str.len() > 0].copy()

vc = df_in_top_k[COL_CODE].value_counts()
clases_objetivo = vc.head(TOP_K_IN).index.tolist()

print(f"[INFO] IN classes: {len(clases_objetivo)} | cobertura aprox: {(vc.head(TOP_K_IN).sum()/vc.sum()):.4f}")

pd.Series(clases_objetivo, name="clases_objetivo").to_csv(path_clases_in, index=False)
print(f"[OK] Guardado: {path_clases_in}")

df_in_top_k["label_in"] = df_in_top_k[COL_CODE].isin(clases_objetivo).astype(int)
print(df_in_top_k["label_in"].value_counts(dropna=False))
print(df_in_top_k.shape)
print(df_in_top_k.columns)


**Notas**:
- TOP_K_IN=430 cubre 80.39% del universo: buen punto si para calidad y cobertura.
- Distribución label_in: 
  - IN 4,119,753 (80.4%) vs OUT 1,005,272 (19.6%). 
  - Por el momento desbalanceado; si se entrena el filtro binario sin balanceo, tenderá a predecir IN.
  - Tamaño final: 5,125,025 filas y 9 columnas (incluye len_descripcion y label_in).

Limpieza y submuestreo con balance

In [ ]:
# Balance dentro de IN + OUT 50/50

df_clean = df_in_top_k.copy()
df_clean[COL_TEXT] = df_clean[COL_TEXT].fillna("").astype(str).str.strip()
df_clean = df_clean[df_clean[COL_TEXT].str.len() > 0].copy()

df_in = df_clean[df_clean["label_in"] == 1]
df_out = df_clean[df_clean["label_in"] == 0]

# Balancear IN por clase HS
df_in_bal = df_in.groupby(COL_CODE, group_keys=False).apply(
    lambda x: x.sample(min(len(x), N_PER_CLASS), random_state=SEED)
)

# Balancear OUT al total de IN balanceado
df_out_bal = df_out.sample(n=min(len(df_out), len(df_in_bal)), random_state=SEED)

df_bal = pd.concat([df_in_bal, df_out_bal]).sample(frac=1, random_state=SEED).reset_index(drop=True)

print("[INFO] IN balanceado:", len(df_in_bal))
print("[INFO] OUT balanceado:", len(df_out_bal))
print(df_bal["label_in"].value_counts())
print("[INFO] Clases IN:", df_in_bal[COL_CODE].nunique())


**Notas**:
- Balance ok: 860k IN y 860k OUT.
- 430 clases IN con ~2,000 muestras por clase (860k / 430).
- Dataset final balanceado (1.72M filas) listo para split/entrenamiento.

Split train/val/test estratificado

In [ ]:
# Split por índices (menos copias)
idx = df_bal.index.to_numpy()
y = df_bal["label_in"].to_numpy()

idx_trainval, idx_test = train_test_split(
    idx, test_size=TEST_SIZE, stratify=y, random_state=SEED
)

val_fraction_of_trainval = VAL_SIZE / (1 - TEST_SIZE)
y_trainval = df_bal.loc[idx_trainval, "label_in"].to_numpy()

idx_train, idx_val = train_test_split(
    idx_trainval, test_size=val_fraction_of_trainval,
    stratify=y_trainval, random_state=SEED
)

df_train = df_bal.loc[idx_train].copy()
df_val   = df_bal.loc[idx_val].copy()
df_test  = df_bal.loc[idx_test].copy()

print(f"[INFO] train={len(df_train):,} | val={len(df_val):,} | test={len(df_test):,}")
print("[INFO] Distribución label_in:")
print(" - train:", df_train["label_in"].value_counts(normalize=True).to_dict())
print(" - val:  ", df_val["label_in"].value_counts(normalize=True).to_dict())
print(" - test: ", df_test["label_in"].value_counts(normalize=True).to_dict())


**Notas**:
- El split generó:
  - Tamaños consistentes con 80/10/10 aprox. sobre 1.72M filas.
  - Distribución balanceada (50/50) en train/val/test.

Dataset + Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

class InOutDataset(Dataset):
    def __init__(self, df, text_col, label_col, tokenizer, max_length):
        self.df = df.reset_index(drop=True)
        self.text_col = text_col
        self.label_col = label_col
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if isinstance(idx, list):
            texts = self.df.loc[idx, self.text_col].astype(str).tolist()
            labels = self.df.loc[idx, self.label_col].astype(int).tolist()
            enc = self.tokenizer(
                texts,
                truncation=True,
                max_length=self.max_length,
                padding=False
            )
            enc["labels"] = labels
            return enc

        text = str(self.df.at[idx, self.text_col])
        label = int(self.df.at[idx, self.label_col])
        enc = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding=False
        )
        enc["labels"] = label
        return enc


train_ds = InOutDataset(df_train, COL_TEXT, "label_in", tokenizer, MAX_LENGTH)
val_ds = InOutDataset(df_val, COL_TEXT, "label_in", tokenizer, MAX_LENGTH)
test_ds = InOutDataset(df_test, COL_TEXT, "label_in", tokenizer, MAX_LENGTH)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
print("[INFO] tokenizer:", MODEL_NAME)
print("[INFO] max_length:", MAX_LENGTH)

train_ds = InOutDataset(df_train, COL_TEXT, "label_in", tokenizer, MAX_LENGTH)
val_ds = InOutDataset(df_val, COL_TEXT, "label_in", tokenizer, MAX_LENGTH)
test_ds = InOutDataset(df_test, COL_TEXT, "label_in", tokenizer, MAX_LENGTH)

print("[INFO] sizes -> train:", len(train_ds), "val:", len(val_ds), "test:", len(test_ds))

# Ejemplo rápido
sample = train_ds[0]
print("[INFO] sample keys:", sample.keys())
print("[INFO] sample label:", sample["labels"])
print("[INFO] sample input_ids length:", len(sample["input_ids"]))


In [ ]:
print("[INFO] sample text:", df_train.iloc[0][COL_TEXT])


Modelo BERT

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=2, 
    id2label={0: "OUT", 1: "IN"}, 
    label2id={"OUT": 0, "IN": 1}
)
model.to(device)
print(model)
print(device)
print("[INFO] num_labels:", model.config.num_labels)
print("[INFO] id2label:", model.config.id2label)

num_params = sum(p.numel() for p in model.parameters())
num_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("[INFO] params:", f"{num_params:,}")
print("[INFO] trainable:", f"{num_trainable:,}")


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = (preds == labels).mean()
    return {"accuracy": acc}

print("MPS:", torch.backends.mps.is_available())


```python
# Asegurar que train_ds sea InOutDataset
train_ds = InOutDataset(df_train, COL_TEXT, "label_in", tokenizer, MAX_LENGTH)

from torch.utils.data import Subset

n_test = 1024
sample_ds = Subset(train_ds, range(min(n_test, len(train_ds))))

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
model.eval()

# Subset pequeño para test rápido
n_test = 512
sample_ds = Subset(train_ds, range(min(n_test, len(train_ds))))

batch_sizes = [4, 8, 12, 16, 24, 32, 48, 64, 96, 128]
max_ok = None

for bs in batch_sizes:
    try:
        print(f"[TEST] batch_size={bs} ...", end="")
        loader = DataLoader(sample_ds, batch_size=bs, collate_fn=data_collator)
        batch = next(iter(loader))
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            _ = model(**batch)
        print(" OK")
        max_ok = bs
    except RuntimeError as e:
        print(" FAIL")
        print("Reason:", e)
        break

print("[INFO] max batch size OK:", max_ok)
```

```text
[TEST] batch_size=4 ... OK
[TEST] batch_size=8 ... OK
[TEST] batch_size=12 ... OK
[TEST] batch_size=16 ... OK
[TEST] batch_size=24 ... OK
[TEST] batch_size=32 ... OK
[TEST] batch_size=48 ... OK
[TEST] batch_size=64 ... OK
[TEST] batch_size=96 ... OK
[TEST] batch_size=128 ... OK
[INFO] max batch size OK: 128
```

Entrenamiento BERT binario (Trainer)

In [ ]:

print("transformers.__version__ =", transformers.__version__)
print("TrainingArguments module  =", TrainingArguments.__module__)
print("TrainingArguments file    =", inspect.getsourcefile(TrainingArguments))
print("TrainingArguments init sig=", inspect.signature(TrainingArguments.__init__))


In [ ]:
training_args = TrainingArguments(
    output_dir=str(Path(DIR_MODELS) / "tmp_inout"),

    eval_strategy="no",     
    save_strategy="no",
    logging_strategy="steps",
    logging_steps=100,

    per_device_train_batch_size=BATCH_TRAIN,
    per_device_eval_batch_size=BATCH_EVAL,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,

    num_train_epochs=EPOCHS,
    learning_rate=LR,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,

    dataloader_num_workers=0,
    dataloader_pin_memory=False,

    fp16=False,
    bf16=False,
    
    report_to="none",
    disable_tqdm=False,
    remove_unused_columns=False,
    seed=SEED,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
print(trainer)
print("[INFO] train_dataset:", len(trainer.train_dataset))
print("[INFO] eval_dataset:", len(trainer.eval_dataset))
print("[INFO] batch_size_train:", training_args.per_device_train_batch_size)
print("[INFO] batch_size_eval:", training_args.per_device_eval_batch_size)


Calculo de steps

In [ ]:
# Steps por época (aprox)
train_steps_per_epoch = len(trainer.train_dataset) // (
    training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
)
eval_steps = len(trainer.eval_dataset) // training_args.per_device_eval_batch_size

print("[INFO] train steps/epoch:", train_steps_per_epoch)
print("[INFO] eval steps:", eval_steps)


**Notas:**
- train steps/epoch = 18,812: número de actualizaciones por época (tiempo por época).
- eval steps = 1,343: número de batches en validación (determina cuánto tarda evaluar).

Entrenamiento

In [ ]:
# Evitar paralelismo que a veces rompe kernels en Mac
os.environ["TOKENIZERS_PARALLELISM"] = "false" 
torch.set_num_threads(min(8, os.cpu_count() or 8))

# Vaciar caches
gc.collect()
if torch.backends.mps.is_available():
    try:
        torch.mps.empty_cache()
    except Exception:
        pass

print("[INFO] MPS available:", torch.backends.mps.is_available())
print("[INFO] CUDA available:", torch.cuda.is_available())


In [ ]:
BACKUP_DIR = Path(DIR_MODELS) / "bert_inout_ckpt_final"
TRAIN_OUT_PATH = BACKUP_DIR / "train_output.json"
TRAIN_ARGS_PATH = BACKUP_DIR / "training_args.json"

#  Ajustes de estabilidad para Mac 
if hasattr(trainer, "args") and trainer.args is not None:
    # Evitar multiproceso en dataloaders
    trainer.args.dataloader_num_workers = 0
    trainer.args.dataloader_pin_memory = False

    # Bajar eval batch para reducir peak de memoria (si eval ocurre)
    if getattr(trainer.args, "per_device_eval_batch_size", None) is not None:
        trainer.args.per_device_eval_batch_size = min(int(trainer.args.per_device_eval_batch_size), 16)

    # Logging estable
    trainer.args.disable_tqdm = False
    trainer.args.report_to = "none"

# Si el modelo lo permite: gradient checkpointing reduce memoria (a costa de tiempo)
try:
    trainer.model.gradient_checkpointing_enable()
    trainer.model.config.use_cache = False  # importante para evitar uso extra de memoria
    print("[INFO] gradient_checkpointing habilitado.")
except Exception as e:
    print("[WARN] No se pudo habilitar gradient_checkpointing:", e)


# Limpieza preventiva
gc.collect()
if torch.backends.mps.is_available():
    try:
        torch.mps.empty_cache()
    except Exception:
        pass

if BACKUP_DIR.exists():
    print("[INFO] Modelo FINAL ya existe, cargando...")
    if ("model" in globals()) and ("tokenizer" in globals()):
        print("[INFO] Reutilizando modelo/tokenizer ya cargados en memoria.")
    else:
        tokenizer = AutoTokenizer.from_pretrained(str(BACKUP_DIR), use_fast=True)
        model = AutoModelForSequenceClassification.from_pretrained(str(BACKUP_DIR), low_cpu_mem_usage=True)

    try:
        trainer.model = model
    except Exception:
        pass

    if TRAIN_OUT_PATH.exists():
        with open(TRAIN_OUT_PATH, "r", encoding="utf-8") as f:
            train_metrics = json.load(f)
        print("[INFO] train metrics (loaded):", train_metrics)

else:
    print("[INFO] Starting FINAL training...")
    print("[INFO] epochs:", trainer.args.num_train_epochs)
    print("[INFO] batch_train:", trainer.args.per_device_train_batch_size)
    print("[INFO] grad_accum:", trainer.args.gradient_accumulation_steps)
    print("[INFO] lr:", trainer.args.learning_rate)
    # --- Evitar picos: desactivar evaluación durante train y evaluar después ---
    # Si usted tiene evaluation_strategy="epoch" en training_args, esto provoca eval en cada epoch.
    # Lo desactivamos solo para el train final (safe).
    try:
        trainer.args.evaluation_strategy = "no"
        trainer.args.save_strategy = "no"  # evita guardados intermedios grandes
        print("[INFO] evaluation_strategy='no' durante training (safe).")
    except Exception:
        pass

    train_output = trainer.train()

    print("[INFO] Training done.")
    print("[INFO] train_output.metrics:", train_output.metrics)

    # Guardar artefactos finales
    BACKUP_DIR.mkdir(parents=True, exist_ok=True)
    trainer.save_model(str(BACKUP_DIR))
    tokenizer.save_pretrained(str(BACKUP_DIR))

    with open(TRAIN_OUT_PATH, "w", encoding="utf-8") as f:
        json.dump(train_output.metrics, f, ensure_ascii=False, indent=2)

    with open(TRAIN_ARGS_PATH, "w", encoding="utf-8") as f:
        json.dump(
            {
                "bert_base": getattr(tokenizer, "name_or_path", None),
                "max_length": globals().get("MAX_LENGTH", None),
                "num_train_epochs": trainer.args.num_train_epochs,
                "per_device_train_batch_size": trainer.args.per_device_train_batch_size,
                "per_device_eval_batch_size": trainer.args.per_device_eval_batch_size,
                "gradient_accumulation_steps": trainer.args.gradient_accumulation_steps,
                "learning_rate": trainer.args.learning_rate,
                "weight_decay": trainer.args.weight_decay,
                "warmup_ratio": getattr(trainer.args, "warmup_ratio", None),
                "seed": trainer.args.seed,
                "fp16": getattr(trainer.args, "fp16", False),
                "dataloader_num_workers": trainer.args.dataloader_num_workers,
                "gradient_checkpointing": True,
            },
            f,
            ensure_ascii=False,
            indent=2
        )

    print(f"[OK] Modelo FINAL guardado en: {BACKUP_DIR}")
    print(f"[OK] Métricas guardadas en: {TRAIN_OUT_PATH}")
    print(f"[OK] Training args guardados en: {TRAIN_ARGS_PATH}")
